In [1]:
from nltk.tokenize import word_tokenize
import pandas as pd
import spacy 
import matplotlib.pyplot as plt
import gzip,os,glob
import numpy as np
import pickle
import nltk
import codecs
from langdetect import detect

In [2]:
# load data from pkl file 
df = pd.read_pickle('../data/data_filtered contents.pkl')
print(df.shape)

(20247, 5)


In [3]:
# Adds new collumn, detects language of column
df['Languagereview'] = df['content'].apply(detect)

In [4]:
# delets all rows with the value "en"
df = df[df.Languagereview == "de"]

In [5]:
# resets index from 0, 4, 11, ... (random) to 0,1,2, ...
df.reset_index(inplace=True,drop=True)
print(df.shape)

(18143, 6)


In [6]:
# Removing non-alphabetical symbols in dataframe, keeping umlauts
df['content'] = df['content'].replace('[^a-zA-Z \äöüß]', '', regex=True).str.replace(' +',' ').str.strip()
# Transform umlauts
df['content'] = df['content'].str.replace(chr(252), 'ue')
df['content'] = df['content'].str.replace(chr(228), 'ae')
df['content'] = df['content'].str.replace(chr(246), 'oe')
df['content'] = df['content'].str.replace(chr(196), 'Ae')
df['content'] = df['content'].str.replace(chr(214), 'Oe')
df['content'] = df['content'].str.replace(chr(220), 'Ue')
df['content'] = df['content'].str.replace(chr(223), 'ss')
# Lowercase
df['content'] = df['content'].str.lower()
df.head()

,author,title,class,content,word_count,Languagereview
0,NaN,Aernout Mik - Communitas,[Kunstgeschichte],aernout mik zaehlt zu den wichtigsten kuenstle...,64,de
1,NaN,Anton Henning : Dt. /Engl.,NaN,die vorliegende publikation widmet sich drei o...,113,de
2,"Specht, Heike","Die Feuchtwangers : Familie, Tradition und j...","[Sprachwissenschaften, Geschichte]",jahre geschichte deutschjuedischen buergertums...,169,de
3,"Scruton, Roger",Grüne Philosophie : Ein konservativer Denkan...,"[Allgemeines, Philosophie, Theologie und Relig...",der umweltschutz liegt herkoemmlich im hoheits...,101,de
4,"Schlagowski, Heinz",Technische Dokumentation im Maschinen- und An...,[Technik],das buch gibt dem anwender die entscheidenden ...,156,de


<h1>Remove Stop Words</h1>

In [7]:
from nltk.corpus import stopwords
stop = stopwords.words('german')
df['content_nltk'] = df['content'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['content_nltk'].iloc[0]

'aernout mik zaehlt wichtigsten kuenstlern niederlande installationen film video performance skulptur architektur oszillieren kunst bewegten bild grundlegend weiterentwickelt werke spiegeln gesellschaften kriege wirtschaftliche depression wider katalog praesentiert wichtige arbeiten kuenstlers nie zuvor deutschland sehen ausstellung oktober januar museum folkwang essen'

In [8]:
df_test = df[:5]
df_test

,author,title,class,content,word_count,Languagereview,content_nltk
0,NaN,Aernout Mik - Communitas,[Kunstgeschichte],aernout mik zaehlt zu den wichtigsten kuenstle...,64,de,aernout mik zaehlt wichtigsten kuenstlern nied...
1,NaN,Anton Henning : Dt. /Engl.,NaN,die vorliegende publikation widmet sich drei o...,113,de,vorliegende publikation widmet drei ortsbezoge...
2,"Specht, Heike","Die Feuchtwangers : Familie, Tradition und j...","[Sprachwissenschaften, Geschichte]",jahre geschichte deutschjuedischen buergertums...,169,de,jahre geschichte deutschjuedischen buergertums...
3,"Scruton, Roger",Grüne Philosophie : Ein konservativer Denkan...,"[Allgemeines, Philosophie, Theologie und Relig...",der umweltschutz liegt herkoemmlich im hoheits...,101,de,umweltschutz liegt herkoemmlich hoheitsgebiet ...
4,"Schlagowski, Heinz",Technische Dokumentation im Maschinen- und An...,[Technik],das buch gibt dem anwender die entscheidenden ...,156,de,buch gibt anwender entscheidenden informatione...


<h1>Tokenizing</h1>

In [9]:
# df_tok = df['content_nltk']
# df_tok

In [10]:
# tokenized = [ nltk.word_tokenize( str(sentence), language='german' ) for sentence in df_tok ]
# print(tokenized)

In [11]:
# df_tok = df.apply(lambda row: nltk.word_tokenize(row['content_nltk'], language='german'), axis=1)

In [12]:
#df_tok

In [13]:
# df_tok_test = df_tok[:3]
# df_tok_test

<h1>Lemmetization</h1>

In [14]:
import spacy
nlp = spacy.load("de_core_news_sm")

df["content_nltk"] = df["content_nltk"].apply(lambda row: " ".join([w.lemma_ for w in nlp(row)]))

In [15]:
df["content_nltk"]

0        aernout mik zaehlt wichtig kuenstlern niederla...
1        vorliegend publikation widmen drei ortsbezogen...
2        jahre geschichte deutschjuedischen buergertums...
3        umweltschutz liegen herkoemmlich hoheitsgebiet...
4        buch geben anwender entscheidend informationen...
                               ...                        
18138    grundlegende einfuehrung analysis wenden infor...
18139    wasserknappheit zentral gesellschaftlich herau...
18140    programmieren lernen leicht machen setzen wirk...
18141    dokudrama dokufiktion living history dokusoap ...
18142    zeitgenoessische dokumentarische theater richt...
Name: content_nltk, Length: 18143, dtype: object

<h1>Export</h1>

In [16]:
df_content = df['content_nltk']

In [17]:
df.to_pickle('../data/after_nltk/data_after_nltk.pkl')
df_content.to_csv('../data/after_nltk/df_content.csv', index=True)
df.to_excel('../data/after_nltk/data_after_nltk.xlsx', index = False)

In [18]:
# #counts number of lines
# fname = "df_content.txt"
# count = 0
# with open(fname, 'r') as f:
#     for line in f:
#         count += 1
# print("Total number of lines is:", count)